In [2]:
import pandas as pd
import numpy as np
import wget

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 50)

import xgboost as xgb
import sklearn

print("XGB Version          : ", xgb.__version__)
print("Scikit-Learn Version : ", sklearn.__version__)

XGB Version          :  2.1.2
Scikit-Learn Version :  1.4.2


In [56]:
data0 = pd.read_csv("cleaned_verified_online.csv")
data0.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8855181,https://sparkflaretrust.com/connect.html,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:46:02+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
1,8855180,https://llltreuirzleeno.webflow.io/,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:37:15+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
2,8855179,http://llltreuirzleeno.webflow.io,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:37:14+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
3,8855176,http://hubtrezorstart.webflow.io,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:32:28+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
4,8855177,https://hubtrezorstart.webflow.io/,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:32:28+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other


In [57]:
data0.shape


(67678, 8)

In [58]:
phishurl = data0.sample(n = 5000, random_state = 12).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8717279,https://telegra.ph/BT-Business-Communications-...,http://www.phishtank.com/phish_detail.php?phis...,2024-08-16T15:47:59+00:00,yes,2024-08-16T15:53:35+00:00,yes,Other
1,8376431,https://new.express.adobe.com/webpage/pZDKTv9g...,http://www.phishtank.com/phish_detail.php?phis...,2023-11-27T15:31:47+00:00,yes,2023-11-27T15:43:19+00:00,yes,Other
2,8831188,http://suall4ss.top,http://www.phishtank.com/phish_detail.php?phis...,2024-11-01T19:50:22+00:00,yes,2024-11-01T20:02:24+00:00,yes,Other
3,8836530,https://jawatan-kosong.foie.site/04fe663632581...,http://www.phishtank.com/phish_detail.php?phis...,2024-11-06T12:02:16+00:00,yes,2024-11-08T08:51:47+00:00,yes,Other
4,8468077,https://docs.google.com/presentation/d/e/2PACX...,http://www.phishtank.com/phish_detail.php?phis...,2024-02-26T23:11:40+00:00,yes,2024-02-27T10:42:55+00:00,yes,Other


In [59]:
phishurl.shape

(5000, 8)

In [7]:
#legitimate URL
data1 = pd.read_csv("Benign_list_big_final.csv")
data1.columns = ['URLs']
data1.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [8]:
legiurl = data1.sample(n = 5000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

,URLs
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...


In [9]:
legiurl.shape


(5000, 1)

In [10]:
# importing required packages for this section
from urllib.parse import urlparse,urlencode
import ipaddress
import re

In [11]:
# 1.Domain of the URL (Domain) 
def getDomain(url):  
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
	       domain = domain.replace("www.","")
  return domain

In [12]:
# 2.Checks for IP address in URL (Have_IP)
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip

In [13]:
# 3.Checks the presence of @ in URL (Have_At)
def haveAtSign(url):
  if "@" in url:
    at = 1    
  else:
    at = 0    
  return at

In [14]:
# 4.Finding the length of URL and categorizing (URL_Length)
def getLength(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length

In [15]:
# 5.Gives number of '/' in URL (URL_Depth)
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [16]:
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

In [17]:
# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

In [18]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [19]:
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate
     

In [20]:
# 8. Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

In [21]:
!pip install python-whois


In [22]:
# importing required packages for this section
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
import csv
from functools import lru_cache
from datetime import datetime


In [23]:
# 11.DNS Record availability (DNS_Record)
# obtained in the featureExtraction function itself

In [24]:
def web_traffic(url, csv_file):
  # Read CSV data into a dictionary for efficient lookup
  rank_data = {}
  try:
      with open(csv_file, 'r') as csvfile:
          reader = csv.reader(csvfile)
          for row in reader:
              if len(row) >= 2:
                  rank_data[row[1]] = int(row[0])
              else:
                  print(f"Warning: Invalid row in CSV: {row}")
  except FileNotFoundError:
      print(f"CSV file not found: {csv_file}")
      return 0
  except csv.Error as e:
      print(f"Error reading CSV file: {e}")
      return 0

  # Extract domain from URL
  domain = url.split("//")[1].split("/")[0]

  # Check if domain exists in the CSV
  if domain in rank_data:
      rank = rank_data[domain]
      if rank < 100000:
          return 1
      else:
          return 0
  else:
      # Domain not found in CSV, return 1 if rank is assumed to be low
      return 1

In [25]:
# 12.Web traffic (Web_Traffic)
def check_rank(url):
  try:
    #Filling the whitespaces in the URL if any
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 1
  else:
    return 0

In [26]:
# 13.Survival time of domain: The difference between termination time and creation time (Domain_Age)  
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1
    else:
      age = 0
  return age

In [27]:
# 14.End time of domain: The difference between termination time and current time (Domain_End) 
def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0
    else:
      end = 1
  return end

In [28]:
#HTML and JavaScript based Features
import requests

In [29]:
# 16.Checks the effect of mouse over on status bar (Mouse_Over)
def mouseOver(response): 
  if response == "" :
    return 1
  else:
    if re.findall("", response.text):
      return 1
    else:
      return 0

In [30]:
# 15. IFrame Redirection (iFrame)
def iframe(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[|]", response.text):
          return 0
      else:
          return 1

In [31]:
# 17.Checks the status of the right click attribute (Right_Click)
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

In [32]:
# 18.Checks the number of forwardings (Web_Forwards)    
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

In [33]:
#cache domain
@lru_cache(maxsize=5000)
def cached_whois_lookup(domain):
    try:
        return whois.whois(domain)
    except:
        return None

In [34]:
pip install asyncwhois

Note: you may need to restart the kernel to use updated packages.


In [35]:
#Function to extract features
def featureExtraction(url,label):
  #print('url', url,'label', label)
  features = []
  #Address bar based features (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))
  
  #Domain based features (4)
  dns = 0
  try:
    domain_name = asyncwhois.lookup(urlparse(url).netloc)
  except:
    dns = 1

  features.append(dns)
  # features.append(web_traffic(url))
  features.append(web_traffic(url,'top-1m.csv'))
  features.append(1 if dns == 1 else domainAge(domain_name))
  features.append(1 if dns == 1 else domainEnd(domain_name))
  
  #HTML & Javascript based features (4)
  try:
    response = requests.get(url,timeout=5)  
  except:
    response = ""
  features.append(iframe(response))
  features.append(mouseOver(response))
  features.append(rightClick(response))
  features.append(forwarding(response))
  features.append(label)
  
  return features

In [36]:
#Legitimate URLs
legiurl.shape

(5000, 1)

In [37]:
#Extracting the feautres & storing them in a list
#import cProfile

legi_features = []
label = 0

for i in range(0, 5000):
    url = legiurl['URLs'][i]
    print(url,i)
    #cProfile.run("legi_features.append(featureExtraction(url,label))")
    legi_features.append(featureExtraction(url,label))

http://graphicriver.net/search?date=this-month&length_max=&length_min=&price_max=&price_min=&rating_min=&sales=&sort=sales&term=&view=list 0
http://ecnavi.jp/redirect/?url=http://www.cross-a.net/x.php?id=1845_3212_22061_26563&m=1004&pid=%user_id% 1
https://hubpages.com/signin?explain=follow+Hubs&url=%2Fhub%2FComfort-Theories-of-Religion 2
http://extratorrent.cc/torrent/4190536/AOMEI+Backupper+Technician+%2B+Server+Edition+2.8.0+%2B+Patch+%2B+Key+%2B+100%25+Working.html 3
http://icicibank.com/Personal-Banking/offers/offer-detail.page?id=offer-ezeego-domestic-airtravel-20141407112611060 4
http://nypost.com/2015/05/07/us-indifference-leaves-saudis-partnering-with-terrorists/ 5
http://kienthuc.net.vn/diem-thi/diem-chuan-dh-cong-nghe-giao-thong-van-tai-nam-2014-482407.html 6
http://thenextweb.com/in/2015/04/16/india-wants-a-neutral-web-and-facebooks-internet-org-cant-be-a-part-of-it/gtm.js 7
http://tobogo.net/cdsb/board.php?board=greet&bm=view&no=5716&category=&auth=&page=1&search=&keyword=

In [38]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,graphicriver.net,0,0,1,1,0,0,0,0,1,1,1,1,0,1,1,0,0
1,ecnavi.jp,0,0,1,1,1,0,0,0,1,1,1,1,0,1,1,0,0
2,hubpages.com,0,0,1,1,0,0,0,0,1,1,1,1,0,1,1,0,0
3,extratorrent.cc,0,0,1,3,0,0,0,0,1,1,1,1,1,1,1,1,0
4,icicibank.com,0,0,1,3,0,0,0,0,1,1,1,1,1,1,1,0,0


In [41]:
legitimate.to_csv('legitimate.csv', index= False)


In [42]:
phishurl.shape


(5000, 8)

In [60]:
#Extracting the feautres & storing them in a list
phish_features = []
label = 1
for i in range(0, 5000):
  url = phishurl['url'][i]
  print(url,i)
  phish_features.append(featureExtraction(url,label))
     

https://telegra.ph/BT-Business-Communications-08-16 0
https://new.express.adobe.com/webpage/pZDKTv9gdBU1W 1
http://suall4ss.top 2
https://jawatan-kosong.foie.site/04fe6636325819/registration.php 3
https://docs.google.com/presentation/d/e/2PACX-1vQqfA8KTZpL876qRJlO-scPzoo3Gc0h8PrE9ietG6IegpDolmqscjHK3-M977pbndgY_87Ni_3DIvOV/pub?start=false&amp;loop=false&amp;delayms=3000 4
https://importmelo.pe/ 5
https://dfsfger.xyz/ 6
http://newburycafe.co.za/Webmail/57/Webmail/webmail.php 7
https://canada.postcanadaevkwk.top/ 8
http://cuscatlansv9580.webcindario.com/ 9
https://canada.postcanadabwh.top/ 10
https://t.co/P3mbhUs7YX 11
https://gsfsfstssdu.weebly.com/ 12
https://auth-xfinityverification.faqserv.com/xfinity/login/ 13
https://fiffr-12d16.firebaseapp.com/ 14
http://terra.mailcomvoce.com/09stt/acesso/seguro/terra/page/home.php 15
https://bdvpromociona.com/bcp 16
https://pub-b72cd462909d46f0a9b08afad77e852f.r2.dev/serbia.html#noreply@fadata.eu 17
https://docs.google.com/presentation/d/e/2PACX-

In [61]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,telegra.ph,0,0,0,1,0,0,0,0,1,1,1,1,1,1,1,0,1
1,new.express.adobe.com,0,0,0,2,0,0,0,0,1,1,1,1,0,1,1,0,1
2,suall4ss.top,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1
3,jawatan-kosong.foie.site,0,0,1,2,0,0,0,1,1,1,1,1,1,1,1,1,1
4,docs.google.com,0,0,1,5,0,0,0,0,1,1,1,1,0,1,1,0,1


In [62]:
# Storing the extracted legitimate URLs fatures to csv file
phishing.to_csv('phishing.csv', index= False)


In [63]:
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.head()


,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,graphicriver.net,0,0,1,1,0,0,0,0,1,1,1,1,0,1,1,0,0
1,ecnavi.jp,0,0,1,1,1,0,0,0,1,1,1,1,0,1,1,0,0
2,hubpages.com,0,0,1,1,0,0,0,0,1,1,1,1,0,1,1,0,0
3,extratorrent.cc,0,0,1,3,0,0,0,0,1,1,1,1,1,1,1,1,0
4,icicibank.com,0,0,1,3,0,0,0,0,1,1,1,1,1,1,1,0,0


In [64]:
urldata.tail()


,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
9995,vero-fi-bfb14.firebaseapp.com,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1
9996,easyweb-4408b.web.app,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1
9997,yksduhl.cn,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1
9998,pub-843b01bcc2674789907bac533c0f748b.r2.dev,0,0,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1
9999,487387483-minem-gob-pe.weebly.com,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,0,1


In [65]:
urldata.shape


(10000, 18)

In [66]:
# Storing the data in CSV file
urldata.to_csv('urldata.csv', index=False)
